In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
url='https://www.meteo-paris.com/ile-de-france/previsions'

In [3]:
r=requests.get(url)
status = r.status_code

if status==200:
    print('OK')
elif status==404:
    print("Error")
elif status==403:
    print("Access forbidden")
elif status==500:
    print("Internal server error")

OK


In [4]:
s = BeautifulSoup(r.content,'html.parser')

In [5]:
div_pred_temp = []
for text in s.find_all('div',{'class':'forecast-line__pictos--temp'}):
    b = text
    div_pred_temp.append(b)
    
print(div_pred_temp)

[<div class="forecast-line__pictos--temp bg-plus-7">7</div>, <div class="forecast-line__pictos--temp bg-plus-15">15</div>, <div class="forecast-line__pictos--temp bg-plus-5">5</div>, <div class="forecast-line__pictos--temp bg-plus-14">14</div>, <div class="forecast-line__pictos--temp bg-plus-3">3</div>, <div class="forecast-line__pictos--temp bg-plus-15">15</div>, <div class="forecast-line__pictos--temp bg-plus-5">5</div>, <div class="forecast-line__pictos--temp bg-plus-8">8</div>, <div class="forecast-line__pictos--temp bg-plus-1">1</div>, <div class="forecast-line__pictos--temp bg-plus-7">7</div>, <div class="forecast-line__pictos--temp bg-minus-0">0</div>, <div class="forecast-line__pictos--temp bg-plus-9">9</div>, <div class="forecast-line__pictos--temp bg-plus-3">3</div>, <div class="forecast-line__pictos--temp bg-plus-11">11</div>, <div class="forecast-line__pictos--temp bg-plus-4">4</div>, <div class="forecast-line__pictos--temp bg-plus-9">9</div>, <div class="forecast-line__pic

In [6]:
from datetime import date

today = date.today().strftime("%d/%m/%Y")
pred_temp = [pt.get_text() for pt in div_pred_temp][2:]
pred_temp.insert(0,today)
print(pred_temp)

['02/04/2021', '5', '14', '3', '15', '5', '8', '1', '7', '0', '9', '3', '11', '4', '9', '1', '8']


In [7]:
from io import StringIO

url_data='https://static.meteo-paris.com/station/downld02.txt'
r_data=requests.get(url_data)

aux = StringIO(r_data.text)
col_names = ['Date','Time','Temp_Out','Hi_Temp','Low_Temp','Out_Hum','Dew_Pt','Wind_Speed',
            'Wind_Dir','Wind_Run','Hi_Speed','Hi_Dir','Wind_Chill','Heat_Index','THW_Index','Bar',
            'Rain','Rain_Rate','Heat_DD','Cool_DD','In_Temp','In_Hum','In_Dew','In_Heat',
             'In_EMC','In_Air_Dens','Temp_2nd','Wind_Samp','Wind_TX','ISS_Recept','Arc_Int']
df_list = pd.read_csv(aux,skiprows=3,delimiter="\s+",header=None,names=col_names)
df_list.head()

,Date,Time,Temp_Out,Hi_Temp,Low_Temp,Out_Hum,Dew_Pt,Wind_Speed,Wind_Dir,Wind_Run,...,In_Hum,In_Dew,In_Heat,In_EMC,In_Air_Dens,Temp_2nd,Wind_Samp,Wind_TX,ISS_Recept,Arc_Int
0,1/04/21,0:01,19.5,19.5,19.5,45,7.2,0.0,---,0.0,...,28,7.4,26.3,5.72,1.1711,---,23,1,100.0,1
1,1/04/21,0:02,19.5,19.5,19.5,45,7.2,0.0,---,0.0,...,28,7.4,26.3,5.72,1.1712,---,23,1,100.0,1
2,1/04/21,0:03,19.5,19.5,19.5,45,7.2,0.0,---,0.0,...,28,7.5,26.4,5.71,1.1706,---,24,1,100.0,1
3,1/04/21,0:04,19.4,19.5,19.4,45,7.2,0.0,---,0.0,...,28,7.6,26.6,5.71,1.1701,---,22,1,95.7,1
4,1/04/21,0:05,19.4,19.4,19.4,45,7.2,0.0,---,0.0,...,28,7.6,26.6,5.71,1.1701,---,23,1,100.0,1


In [8]:
df_list[df_list['Date']=='28/03/21']['Temp_Out'].max()

nan

In [9]:
from datetime import datetime,timedelta

yesterday = datetime.now() - timedelta(1)
datetime.strftime(yesterday, '%Y-%m-%d')

datetime.strptime('28/03/21 0:02', '%d/%m/%y %H:%M')

datetime.datetime(2021, 3, 28, 0, 2)

In [10]:
#for d in df_list.iterrows():
    #datetime.strptime(d[1]['Date'], '%d/%m/%y')
    
pd.to_datetime(df_list['Date'] + ' ' + df_list['Time'])

#df_list['Date_format']=datetime.strptime(df_list['Date'], '%d/%m/%y')

0      2021-01-04 00:01:00
1      2021-01-04 00:02:00
2      2021-01-04 00:03:00
3      2021-01-04 00:04:00
4      2021-01-04 00:05:00
               ...        
2669   2021-02-04 20:30:00
2670   2021-02-04 20:31:00
2671   2021-02-04 20:32:00
2672   2021-02-04 20:33:00
2673   2021-02-04 20:34:00
Length: 2674, dtype: datetime64[ns]

In [11]:
### NOAA documentation
### https://www.ncdc.noaa.gov/cdo-web/webservices/v2#gettingStarted
### dataset='https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets'

In [79]:
#Most diffifcult part is to get the GHCND code for locations

dataset='GHCND'
location="GHCND:FR000007150" #Paris Montsouris
location="GHCND:FRM00007149" #Paris Orly
location="GHCND:EI000003969" #Dublin Phoenix Park
n_token='VMPMXiToPnVyavbJBWucazFqvkfkQnAD'
s_date = '2000-03-01'
e_date = '2000-10-01'

weathers_url="https://www.ncdc.noaa.gov/cdo-web/api/v2/data/"
params='stationid=' + location + '&' 'datasetid=' + dataset + '&' + 'startdate=' + s_date + '&' + 'enddate=' + e_date + '&' + 'limit=1000' + '&' + 'units=metric'# + '&datatypeid=TMAX'
noaa_token = {'token': n_token}

req = requests.get(weathers_url, params=params, headers=noaa_token)
df = pd.DataFrame(req.json()['results'])
df.head(10)

,date,datatype,station,attributes,value
0,2000-03-01T00:00:00,PRCP,GHCND:EI000003969,",,E,",1.1
1,2000-03-01T00:00:00,TAVG,GHCND:EI000003969,"H,,S,",4.1
2,2000-03-01T00:00:00,TMAX,GHCND:EI000003969,",,E,",9.5
3,2000-03-01T00:00:00,TMIN,GHCND:EI000003969,",,E,",0.9
4,2000-03-02T00:00:00,PRCP,GHCND:EI000003969,",,E,",6.7
5,2000-03-02T00:00:00,TAVG,GHCND:EI000003969,"H,,S,",7.9
6,2000-03-02T00:00:00,TMAX,GHCND:EI000003969,",,E,",11.0
7,2000-03-02T00:00:00,TMIN,GHCND:EI000003969,",,E,",2.5
8,2000-03-03T00:00:00,PRCP,GHCND:EI000003969,",,E,",0.1
9,2000-03-03T00:00:00,TAVG,GHCND:EI000003969,"H,,S,",4.8
